## Sensitivity Mapping

### Key Idea: Where are lesions that cause X connected to?

### Mechanics:
- 1. Stack all the lesion networks for a given symptom type
- 2. Apply a threshold to the stack of lesion networks to binarize them at a given level
- 3. At **each voxel**, calculate the percentage of lesion networks that survived the threshold
- 4. Store these percentage values themselves as a 3d image.

In [9]:
import numpy as np
from nilearn.maskers import NiftiMasker
import pandas as pd

# This function is just numpy!

def calculate_sensitivity(data, threshold, percent=False, two_tailed=True):
    """
    The goal of this function is to take in a 2d array of data (n_subject, n_image_points)
    and return a 1d array of the percentage of subjects that have a value greater than the threshold.
    We do this in a two-sided fashion. If a subject has a value greater than the threshold, we assign it a 1. 
    If a subject has a value less than the negative threshold, we assign it a -1.
    We then take the mean of the positive and negative values across subjects.
    Finally, we take the maximum of the absolute value of the positive and negative means to determine which value to keep at each point.
    """

    # Threshold/binarize the stack of lesion networks _above_ the threshold
    data_positive = np.where(data > threshold, 1, 0)

    # Threshold/binarize the stack of lesion networks _below_ the negative threshold
    data_negative = np.where(data < - threshold, -1, 0)

    # Calculate the proportion of subjects that have a value greater than the threshold
    data_positive_mean = np.mean(data_positive, axis=0)

    # Calculate the proportion of subjects that have a value less than the negative threshold
    data_negative_mean = np.mean(data_negative, axis=0)

    if two_tailed:
        data_overlap = np.where(np.abs(data_positive_mean) > np.abs(data_negative_mean), data_positive_mean, data_negative_mean)
    else:
        data_overlap = data_positive_mean
    if percent:
        data_overlap = data_overlap * 100
    return np.squeeze(data_overlap)


df = pd.read_csv('example_data/example_dataset.csv')
masker = NiftiMasker(mask_img='example_data/MNI152_T1_2mm_brain_mask.nii.gz').fit()

data = masker.transform(df['t'])

sensitivity = calculate_sensitivity(data, 5, percent=True, two_tailed=True)

sensitivity_img = masker.inverse_transform(sensitivity)
sensitivity_img.to_filename('example_data/sensitivity_example.nii.gz')